In [115]:
#!/usr/bin/env python
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
import numpy as np
import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
import itertools
import datetime
import sys

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 8).set("spark.driver.maxResultSize", "4g")
sc = SparkContext()
hc = HiveContext(sc)
sc.setCheckpointDir('checkpoint/')

In [ ]:
train.head()

In [ ]:
import os
import numpy as np
import sklearn
from sklearn import datasets, metrics, model_selection
from pylightgbm.models import GBMClassifier

exec_path = "/data1/share/LightGBM/lightgbm"

X, Y = datasets.make_classification(n_samples=1000, n_features=10)

gbm = GBMClassifier(exec_path=exec_path,
                    metric='binary_error', early_stopping_round=10, bagging_freq=10)

param_grid = {'learning_rate': [0.1, 0.04], 'bagging_fraction': [0.5, 0.9]}

scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True)
clf = model_selection.GridSearchCV(gbm, param_grid, scoring=scorer, cv=2)

clf.fit(X, Y)

print("Best score: ", clf.best_score_)
print("Best params: ", clf.best_params_)


In [ ]:
section_dict = hc.sql('select * from user_kposminin.yaca_section_dict').toPandas()
section_dict['code'] = section_dict.index

In [49]:
import scipy.sparse
X,y = scipy.sparse.rand(1000,20),np.random.choice([0,1],1000)

In [56]:
import sklearn
import xgboost as xgb

clfXGB = xgb.XGBClassifier(
 learning_rate = 0.1,
 n_estimators=100,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight = 1,
 seed=27
 )

clfXGB.fit(X,y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7)

In [141]:

train_sample_query = '''
select label,first_day,max_score,feat_1,feat_cnt,feat_max_score,feat_avg_score,feat_avg_tic from user_kposminin.la_yaca_ccall_20161103 
where substr(md5(phone_num),1,1) = '1' or label = 1
'''

train = hc.sql(train_sample_query) \
    .toPandas()

In [132]:
test_sample = hc.sql('''select label,first_day,max_score,feat_1,feat_cnt,feat_max_score,feat_avg_score,feat_avg_tic 
   from user_kposminin.la_yaca_ccall_20161108 where substr(md5(phone_num),1,1) = "#k"'''.replace('#k','5')) \
    .toPandas()

In [142]:
#sd = section_dict[['section_ind','code']].values.tolist()

with open('/data1/share/kosm/data/train_1.txt','w') as f:
    for l,feat in train[train['feat_1'] != ''][['label','feat_1']].values:
        for r in sd:
            feat = feat.replace(r[0],str(r[1]))
        f.write('{} {}\n'.format(l,feat))

#with open('/data1/share/kosm/data/test_1.txt','w') as f:
#    for l,feat in test_sample[test_sample['feat_1'] != ''][['label','feat_1']].values:
#        for r in sd:
#            feat = feat.replace(r[0],str(r[1]))
#        f.write('{} {}\n'.format(l,feat))

In [143]:
#! head -5 /data1/share/kosm/data/test_1.txt
! /data1/share/LightGBM/lightgbm  config=/home/k.osminin/lgbm_files/train.conf num_trees=500 num_leaves=127 \
  data=/data1/share/kosm/data/train.txt valid=/data1/share/kosm/data/test.txt > /home/k.osminin/lgbm_files/yaca_lgbm.log
! cat /home/k.osminin/lgbm_files/train.conf /home/k.osminin/lgbm_files/yaca_lgbm.log > /home/k.osminin/lgbm_files/yaca_lgbm_1.log
! cat /home/k.osminin/lgbm_files/yaca_lgbm_1.log

# task type, support train and predict
task = train

# boosting type, support gbdt for now, alias: boosting, boost
boosting_type = gbdt

# application type, support following application
# regression , regression task
# binary , binary classification task
# lambdarank , lambdarank task
# alias: application, app
objective = binary

# eval metrics, support multi metric, delimite by ',' , support following metrics
# l1 
# l2 , default metric for regression
# ndcg , default metric for lambdarank
# auc 
# binary_logloss , default metric for binary
# binary_error
metric = auc,binary_logloss

# frequence for metric output
metric_freq = 1

# true if need output metric for training data, alias: tranining_metric, train_metric
is_training_metric = true

# number of bins for feature bucket, 255 is a recommend setting, it can save memories, and also has good accuracy. 
max_bin = 255

# training data
# if exsting weight file, should name to "binary.train.weight"
# alias: train_data, train
#data = bi

In [135]:
#Write full test file and rename columns to make it dense
with open('/data1/share/kosm/data/full_test.txt','w') as f:
    for k in list('0123456789abcdef'):
        batch = hc.sql('''select label,feat_1 from user_kposminin.la_yaca_ccall_20161108 
                      where substr(md5(phone_num),1,1) = "#k"'''.replace('#k',k)) \
        .collect()
        for l,feat in batch:
            if len(feat)>5:
                for r in sd:
                    feat = feat.replace(r[0],str(r[1]))
                f.write('{} {}\n'.format(l,feat))
    


In [136]:
! /data1/share/LightGBM/lightgbm  config=/home/k.osminin/lgbm_files/test.conf input_model=lgbm.model \
   data=/data1/share/kosm/data/full_test.txt output_result=/data1/share/kosm/data/full_test_predict.txt
#! perl -ne 'print substr($_,0,1)' /data1/share/kosm/data/full_test.txt > /data1/share/kosm/data/full_test_label.txt
os.popen('''perl -ne 'print substr($_,0,1)' /data1/share/kosm/data/full_test.txt > /data1/share/kosm/data/full_test_label.txt''').read()

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 500 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


''

In [137]:
scores = [float(e.strip()) for e in open('/data1/share/kosm/data/full_test_predict.txt','r')]
labels = [int(e) for e in open('/data1/share/kosm/data/full_test_label.txt','r').read()]
len(scores),len(labels)


(18841723, 18841723)

In [138]:

import sklearn.metrics as m
auc = m.roc_auc_score(y_true = labels, y_score = scores)
print('Full test ROC AUC {}.'.format(auc))

Full test ROC AUC 0.576667483767.


In [139]:
data = sorted(zip(scores,labels),reverse = True)
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for XGBoost\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
#pos_tot = sum([e[1] for e in data])
#pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for XGBoost

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	52        	1.34%     	1884      	2.76008%  	133.09    	0.99224   
0.9998    	75        	1.94%     	3768      	1.99045%  	95.98     	0.98752   
0.9997    	111       	2.87%     	5652      	1.96391%  	94.70     	0.97843   
0.9995    	155       	4.00%     	9420      	1.64544%  	79.34     	0.96810   
0.999     	185       	4.78%     	18841     	0.98190%  	47.35     	0.95831   
0.998     	255       	6.59%     	37683     	0.67670%  	32.63     	0.92963   
0.997     	294       	7.59%     	56525     	0.52012%  	25.08     	0.91489   
0.995     	365       	9.43%     	94208     	0.38744%  	18.68     	0.86838   
0.99      	476       	12.29%    	188417    	0.25263%  	12.18     	0.80575   
0.98      	631       	16.30%    	376834    	0.16745%  	8.07      	0.74449   
0.95      	1017      	26.27%    	942086    	0.10795%  	5.21      	0.64703   
0.9       	1573      	40.62%    	1884172   	0.08348

In [ ]:

label = 'label'
feat_col = 'feat_1'
clf = GBMClassifier(exec_path = exec_path, min_data_in_leaf=1)
clf.fit(train[feat_col], train[label], test_data=[(test_sample[feat_col], test_sample[label])])

In [ ]:
feat_cols = [u'max_tcs_score',
       u'max_work_hours_tcs_score', u'avg_work_hours_tcs_score', u'cnt',
       u'hits', u'ip_cnt', u'page_cnt', u'id_cnt', u'work_hours_hits',
       u'min_avg_hour', u'max_avg_hour', u'avg_avg_hour', u'sum_duration',
       u'avg_duration', u'emailru', u'mobile_share', u'vk_share',
       u'social_share', u'hour_std', u'log_avg_exp_tsc_score',
       u'avg_tcs_score', u'sum_tcs_score', u'min_tcs_score', u'q75_tcs_score',
       u'q25_tcs_score', u'q50_tcs_score', u'q90_tcs_score',
       u'good_tcs_urlfr_visited_cnt', u'very_good_tcs_urlfr_visited_cnt',
       u'max_ref_search_score', u'avg_ref_search_score',
       u'q90_ref_search_score', u'cnt_ref_search', u'max_urlfr_avg_hour',
       u'max_urlfr_hits_cnt', u'max_urlfr_cnt_total', u'max_ref_social_score',
       u'avg_ref_social_score', u'q90_ref_social_score', u'cnt_ref_social',
       u'max_utm_score', u'avg_utm_score', u'q90_utm_score', u'cnt_utm',
       u'max_emailru_score', u'avg_emailru_score', u'q90_emailru_score',
       u'cnt_emailru', u'max_email_score', u'avg_email_score',
       u'q90_email_score', u'cnt_email']
#X = train[feat_cols]
#y = train['label']

In [ ]:
import sklearn.linear_model
scaler = sklearn.preprocessing.StandardScaler()

imp = sklearn.preprocessing.Imputer(strategy='mean')
X = scaler.fit_transform(imp.fit_transform(train[feat_cols]))
y = train['label']
Xt = scaler.transform(imp.transform(test_sample[feat_cols]))
yt = test_sample['label']


In [ ]:
clfLogRegr = sklearn.linear_model.LogisticRegression(penalty = 'l1')
clfLogRegr.fit(X,y)

In [ ]:
print('{0} train AUCROC: {1}'.format('LogRegr',sklearn.metrics.roc_auc_score(
                y_true = y, 
                y_score = clfLogRegr.predict_proba(X)[:,1]
    )))
print('{0} sample test AUCROC: {1}'.format('LogRegr',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = clfLogRegr.predict_proba(Xt)[:,1]
    )))

In [ ]:
clfSVM = sklearn.svm.SVC()
clfSVM.fit(X,y)

In [ ]:
print('{0} train AUCROC: {1}'.format('clfSVM',sklearn.metrics.roc_auc_score(
                y_true = y, 
                y_score = clfSVM.predict_proba(X)[:,1]
    )))
print('{0} sample test AUCROC: {1}'.format('clfSVM',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = clfSVM.predict_proba(Xt)[:,1]
    )))

In [ ]:
import sklearn
import xgboost as xgb

clfXGB = xgb.XGBClassifier(
 learning_rate = 0.1,
 n_estimators=100,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight = 1,
 seed=27
 )

clfXGB.fit(X,y)

In [ ]:
print('{0} train AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = y, 
                y_score = clfXGB.predict_proba(X)[:,1]
    )))

In [ ]:
test = []
for k in list('0123456789abcdef'):
    batch = hc.sql('select * from user_kposminin.la_many_features_20161103 where substr(md5(phone),1,1) = "#k"'.replace('#k',k)) \
    .toPandas()
    batch['XGB pred'] = clfXGB.predict_proba(batch[feat_cols])[:,1]
    test = test + batch[['label','max_tcs_score','XGB pred']].values.tolist()

In [ ]:
#test['XGB pred'] = clfXGB.predict_proba(test[feat_cols])[:,1]
print('{0} test XGB pred AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = [e[0] for e in test], 
                y_score = [e[2] for e in test]
    )))

In [ ]:
print('{0} test max_tcs_score AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = [e[0] for e in test],
                y_score = [e[1] for e in test]
    )))

In [ ]:
sorted(zip(clfXGB.feature_importances_,feat_cols),reverse=True)

In [ ]:
#sl = sorted(zip(scores,labels),reverse = True)
sl = sorted([[e[2],e[0]] for e in test],reverse = True) # xgb

In [ ]:
data = sl
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for XGBoost\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
#pos_tot = sum([e[1] for e in data])
#pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))

In [ ]:
import math
sl1 = sorted([[e[1] if not math.isnan(e[1]) else -30,e[0]] for e in test],reverse = True)

In [ ]:
data = sl1
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for max score\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))

In [ ]:
#type(sl1[10002][0])
#data = sorted(sl1,key  = lambda e: -e[0])


In [ ]:
pred = [float(v) for v in open('lgbm_files/LightGBM_predict_result.txt','r').read().split('\n')[:-1]]
labels = [int(v) for v in open('/data1/share/kosm/data/labels_la_trunc_feat_20161103_1.txt','r').read()]

In [ ]:
print('{0} test LGBM AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = labels,
                y_score = pred
    )))

In [ ]:
#data = sorted(zip(pred,labels),reverse = True)

def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for LGBM \n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
#pos_tot = sum([e[1] for e in data])
#pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.9979,0.9978,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))

In [ ]:
for i,v in feat_import:
    print(feat_cols[i],v)

In [ ]:
import sklearn
print('{0} test max_tcs_score AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = train['label'],
                y_score = train[u'page_cnt']
    )))

In [ ]:
train.count()